# Discussion 4

### Due Friday Jan 31, 11:59:59PM



---


In [37]:
# import libraries
import pandas as pd
import numpy as np
import os
import disc04 as disc
# helper function for the notebook in util.py
# take a look at the file!
from util import multi_table

# Review: Combining DataFrames 

#### `merge()`

* Used to combine two (or more) dataframes on the basis of **values of common columns** (indices can also be used, use `left_index=True` and/or `right_index=True`).
    * If we are joining columns on columns, the DataFrame indexes will be ignored. 
    * If we are joining indexes on indexes or indexes on a column or columns, the index will be passed on.

In [2]:
# left dataframe
left = pd.DataFrame({
   'id':[1,2,3,4,5],
   'Name': ['Aaron', 'Marina', 'Justin', 'Janine', 'Ilya'],
   'subject_id':['sub1','sub2','sub4','sub6','sub5']})

# right dataframe
right = pd.DataFrame(
   {'id':[1,2,3,4,5],
   'Name': ['Enrique', 'Parker', 'Erik', 'Allston', 'Betty'],
   'subject_id':['sub2','sub4','sub3','sub6','sub5']})

multi_table([left, right])

,id,Name,subject_id
0,1,Aaron,sub1
1,2,Marina,sub2
2,3,Justin,sub4
3,4,Janine,sub6
4,5,Ilya,sub5
,id,Name,subject_id
0,1,Enrique,sub2
1,2,Parker,sub4
2,3,Erik,sub3
3,4,Allston,sub6


* **`on`**: column or index level names to join on. 
    * These must be found in both DataFrames. 
    * If `on` is `None` and not merging on indexes then this defaults to the intersection of the columns in both DataFrames.

In [4]:
# merge left and right tables on 'id' column
on_id = pd.merge(left,right,on='id')

# how many rows, how many columns?
multi_table([left, right, on_id])

,id,Name,subject_id
0,1,Aaron,sub1
1,2,Marina,sub2
2,3,Justin,sub4
3,4,Janine,sub6
4,5,Ilya,sub5
,id,Name,subject_id
0,1,Enrique,sub2
1,2,Parker,sub4
2,3,Erik,sub3
3,4,Allston,sub6


In [5]:
# merge left and right tables on 'id' and 'subject_id' columns
on_id_subject = pd.merge(left,right,on=['id', 'subject_id'])

# how many rows, how many columns, what are the indices?
multi_table([left, right, on_id_subject])

,id,Name,subject_id
0,1,Aaron,sub1
1,2,Marina,sub2
2,3,Justin,sub4
3,4,Janine,sub6
4,5,Ilya,sub5
,id,Name,subject_id
0,1,Enrique,sub2
1,2,Parker,sub4
2,3,Erik,sub3
3,4,Allston,sub6


* **`how`**: specifies how to determine which keys are to be included in the resulting table. 
    * If a key (column name) combination does not appear in either the left or the right tables, the values in the joined table will be `np.NaN`.
    * Defaults to `inner`, joining will be performed on index. 

In [6]:
data_a = {
        'subject_id': ['1', '2', '3', '4', '5'],
        'first_name': ['Manny', 'Will', 'Hunter', 'Ian', 'Eric'], 
        'last_name': ['Machado', 'Myers', 'Renfroe', 'Kinsler', 'Hosmer']}
df_a = pd.DataFrame(data_a, columns = ['subject_id', 'first_name', 'last_name'])

data_b = {
        'subject_id': ['4', '5', '6', '7', '8'],
        'first_name': ['Cody', 'Justin', 'Corey', 'Clayton', 'Kenley'], 
        'last_name': ['Bellinger', 'Turner', 'Seager', 'Kershaw', 'Jansen']}
df_b = pd.DataFrame(data_b, columns = ['subject_id', 'first_name', 'last_name'])

multi_table([df_a, df_b])

,subject_id,first_name,last_name
0,1,Manny,Machado
1,2,Will,Myers
2,3,Hunter,Renfroe
3,4,Ian,Kinsler
4,5,Eric,Hosmer
,subject_id,first_name,last_name
0,4,Cody,Bellinger
1,5,Justin,Turner
2,6,Corey,Seager
3,7,Clayton,Kershaw


| Merge Method  | Description                  |
| :-------      | :---------------------------:| 
| `left`        | Use keys from left object    | 
| `right`       | Use keys from right object   | 
| `outer`       | Use union of keys            |
| `inner`       | Use intersection of keys     | 

In [18]:
# based on the output below, what 'how' argument was passed into pd.merge?
how_list = ['outer', 'inner', 'right', 'left']

merge_method = np.random.choice(how_list)

pd.merge(df_a, df_b, on='subject_id', how=merge_method)

,subject_id,first_name_x,last_name_x,first_name_y,last_name_y
0,4,Ian,Kinsler,Cody,Bellinger
1,5,Eric,Hosmer,Justin,Turner
2,6,NaN,NaN,Corey,Seager
3,7,NaN,NaN,Clayton,Kershaw
4,8,NaN,NaN,Kenley,Jansen


In [19]:
# let's check!
merge_method

'right'

#### `concat()`

* Used to append one (or more) dataframes one below the other (or sideways, depending on whether the axis option is set to 0 or 1).
    * Useful if we have two or more data sets containing the same columns but different rows of data.
    * We can also the columns from one `Dataframe` to those of another `Dataframe`.

In [24]:
# add 'left' below 'right'
pd.concat([right, left])

,id,Name,subject_id
0,1,Enrique,sub2
1,2,Parker,sub4
2,3,Erik,sub3
3,4,Allston,sub6
4,5,Betty,sub5
0,1,Aaron,sub1
1,2,Marina,sub2
2,3,Justin,sub4
3,4,Janine,sub6
4,5,Ilya,sub5


In [25]:
# if you want to keep track of the names dataframes after concat, use 'keys'
pd.concat([right, left], keys=['right', 'left'])

id     Name subject_id
right 0   1  Enrique       sub2
      1   2   Parker       sub4
      2   3     Erik       sub3
      3   4  Allston       sub6
      4   5    Betty       sub5
left  0   1    Aaron       sub1
      1   2   Marina       sub2
      2   3   Justin       sub4
      3   4   Janine       sub6
      4   5     Ilya       sub5

In [28]:
# add 'left' to the side of 'right'
pd.concat([right, left], axis=1)

,val,val
key,,
foo,4,1
bar,5,2


#### `join()`

* Used to merge two dataframes on the basis of the index; instead of using `merge()` with the option `left_index=True` we can use `join()`.
    * Join operation honors the object on which it is called: `a.join(b)` $ \neq $ `b.join(a)`.

<img src="join_types.jpg">

1. **Inner Join** – default behavior, only keep rows where the merge “on” value exists in both the left and right dataframes.
2. **Left Outer** – keep every row in the left dataframe.
    * Where there are missing values of the “on” variable in the right dataframe, add `np.NaN` values in the result.
3. **Right Join** – keep every row in the right dataframe. 
    * Where there are missing values of the “on” variable in the left column, add `np.NaN` values in the result.
4. **Outer Join** – returns all the rows from the left dataframe, all the rows from the right dataframe, and matches up rows where possible, with `NaNs` elsewhere.

We'll start with a simple example:

In [29]:
left = pd.DataFrame({'key': ['foo', 'bar'], 'val': [1, 2]}).set_index('key')
right = pd.DataFrame({'key': ['foo', 'bar'], 'val': [4, 5]}).set_index('key')

joined = left.join(right, lsuffix='_l', rsuffix='_r')

multi_table([left, right, joined])

Now let's try something a bit more complex:

In [85]:
df1_data = {
    'Year' : [2014, 2014, 2014, 2014, 2014],
    'Week' : ['A', 'B', 'B', 'C', 'D'],
    'Color' : ['Red', 'Red', 'Black', 'Red', 'Green'],
    'Val' : [50, 60, 70, 10, 20]
}

df1 = pd.DataFrame(df1_data).set_index('Week')

df2_data = {
    'Year' : [2014, 2014, 2014, 2014, 2014],
    'Week' : ['A', 'B', 'C', 'C', 'D'],
    'Color' : ['Black', 'Black', 'Green', 'Red', 'Red'],
    'Score' : [30, 100, 50, 20, 40]
}

df2 = pd.DataFrame(df2_data).set_index('Week')

multi_table([df1, df2])

,Year,Color,Val
Week,,,
A,2014,Red,50
B,2014,Red,60
B,2014,Black,70
C,2014,Red,10
D,2014,Green,20
,Year,Color,Score
Week,,,
A,2014,Black,30
B,2014,Black,100


In [31]:
# how many rows, how many columns?
df1.join(df2, lsuffix='_l', rsuffix = '_r')

,Year_l,Color_l,Val,Year_r,Color_r,Score
Week,,,,,,
A,2014,Red,50,2014,Black,30
B,2014,Red,60,2014,Black,100
B,2014,Black,70,2014,Black,100
C,2014,Red,10,2014,Green,50
C,2014,Red,10,2014,Red,20
D,2014,Green,20,2014,Red,40


In [32]:
# will this be any different?
df2.join(df1, lsuffix='_l', rsuffix = '_r')

,Year_l,Color_l,Score,Year_r,Color_r,Val
Week,,,,,,
A,2014,Black,30,2014,Red,50
B,2014,Black,100,2014,Red,60
B,2014,Black,100,2014,Black,70
C,2014,Green,50,2014,Red,10
C,2014,Red,20,2014,Red,10
D,2014,Red,40,2014,Green,20


---

## Practice Problems

**Question 1**

You are given two seperate dataframes: `mlb_2017` and `mlb_2018`. Both dataframes contain statistics for the 2017 and 2018 baseball seasons respectively. Your job is two combine these two dataframes into one using the following guidelines:

* The dataframe you return should be indexed by team name (`Tm`).
* The dataframe you return should include all columns from both `mlb_2017` and `mlb_2018`.
* Use the suffixes `_2017` and `_2018` to differentiate between statistics from both seasons.

Create a function `combined_seasons` that returns, as a tuple, the following:

* The combined dataframe described above.
* The team with most homeruns (`HR`) bewteen the 2017 and 2018 seasons combined.

In [95]:
# read in the following .txt files
mlb_2017 = pd.read_csv(os.path.join('data','mlb_2017.txt'))
mlb_2018 = pd.read_csv(os.path.join('data','mlb_2018.txt'))

#multi_table([mlb_2017.head(), mlb_2018.head()])

In [100]:
joined = pd.merge(mlb_2017,mlb_2018, on = 'Tm', suffixes = ('_2017', '_2018')).set_index('Tm')

In [102]:
highest_hr = joined.filter(regex = 'HR').sum(axis = 1).idxmax()

'NYY'

In [58]:
#result = disc.combined_seasons(mlb_2017, mlb_2018)
result

,Tm,#Bat,BatAge,R/G,G,PA,AB,R,H,2B,...,SLG,OPS,OPS+,TB,GDP,HBP,SH,SF,IBB,LOB
0,ARI,45,28.3,5.01,162,6224,5525,812,1405,314,...,0.445,0.774,94,2457,106,54,39,27,44,1118
1,ATL,49,28.7,4.52,162,6216,5584,732,1467,289,...,0.412,0.738,92,2303,137,66,59,32,57,1127
2,BAL,50,28.6,4.59,162,6140,5650,743,1469,269,...,0.435,0.747,100,2458,138,50,10,37,12,1041
3,BOS,49,27.3,4.85,162,6338,5669,785,1461,302,...,0.407,0.736,92,2305,141,53,9,36,48,1134
4,CHC,47,27.1,5.07,162,6283,5496,822,1402,274,...,0.437,0.775,99,2403,134,82,48,32,54,1147
,Tm,#Bat,BatAge,R/G,G,PA,AB,R,H,2B,...,SLG,OPS,OPS+,TB,GDP,HBP,SH,SF,IBB,LOB
0,ARI,49,29.2,4.28,162,6157,5460,693,1283,259,...,0.397,0.707,85,2170,110,52,38,45,36,1086
1,ATL,58,27.3,4.69,162,6251,5582,759,1433,314,...,0.417,0.742,99,2330,99,66,49,43,53,1143
2,BAL,56,28.4,3.84,162,6034,5507,622,1317,242,...,0.391,0.689,90,2153,132,57,13,35,19,1027
3,BOS,44,27.7,5.41,162,6302,5623,876,1509,355,...,0.453,0.792,112,2550,130,55,7,48,38,1124


In [52]:
# joined = mlb_2017.set_index('Tm').join(mlb_2018.set_index('Tm'), lsuffix='_2017', rsuffix='_2018')
# joined.shape

(30, 56)

**Question 2**

Using the same two dataframes, `mlb_2017` and `mlb_2018`, create a function `seasonal_average` that combines them and takes the mean of each column for each team. 

* The dataframe you return should be indexed by team name (`Tm`).
* Each column should contain the average value between the *2017* and *2018* seasons for the given statistic for each team.
    * For example, the `HR` column should contain the average value for `HR` for each team between the *2017* and *2018* seasons.

In [93]:
mlb_2017 = pd.read_csv(os.path.join('data','mlb_2017.txt'))
mlb_2018 = pd.read_csv(os.path.join('data','mlb_2018.txt'))

#pd.concat([mlb_2017, mlb_2018]).groupby('Tm').mean().shape

In [94]:
mlb_2017

,Tm,#Bat,BatAge,R/G,G,PA,AB,R,H,2B,...,SLG,OPS,OPS+,TB,GDP,HBP,SH,SF,IBB,LOB
0,ARI,45,28.3,5.01,162,6224,5525,812,1405,314,...,0.445,0.774,94,2457,106,54,39,27,44,1118
1,ATL,49,28.7,4.52,162,6216,5584,732,1467,289,...,0.412,0.738,92,2303,137,66,59,32,57,1127
2,BAL,50,28.6,4.59,162,6140,5650,743,1469,269,...,0.435,0.747,100,2458,138,50,10,37,12,1041
3,BOS,49,27.3,4.85,162,6338,5669,785,1461,302,...,0.407,0.736,92,2305,141,53,9,36,48,1134
4,CHC,47,27.1,5.07,162,6283,5496,822,1402,274,...,0.437,0.775,99,2403,134,82,48,32,54,1147
5,CHW,51,26.7,4.36,162,6059,5513,706,1412,256,...,0.417,0.731,96,2300,124,76,35,33,17,1055
6,CIN,47,27.1,4.65,162,6213,5484,753,1390,249,...,0.433,0.761,97,2372,116,72,50,42,41,1135
7,CLE,41,28.0,5.05,162,6234,5511,818,1449,333,...,0.449,0.788,104,2476,125,50,23,45,30,1158
8,COL,41,28.3,5.09,162,6201,5534,824,1510,293,...,0.444,0.781,90,2455,143,44,62,41,46,1088
9,DET,49,29.6,4.54,162,6150,5556,735,1435,289,...,0.424,0.748,98,2355,128,52,11,27,21,1104
